In [16]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from datetime import datetime
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, UpSampling2D, Dropout,BatchNormalization,GlobalAveragePooling2D
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [17]:
# ! git config --global --unset https.proxy
! git clone https://github.com/seshuad/IMagenet
! ls 'IMagenet/tiny-imagenet-200/'

import time
import imageio.v3 as nd
import numpy as np

path = 'IMagenet/tiny-imagenet-200/'

def get_id_dictionary():
    id_dict = {}
    for i, line in enumerate(open( path + 'wnids.txt', 'r')):
        id_dict[line.replace('\n', '')] = i
    return id_dict

def get_class_to_id_dict():
    id_dict = get_id_dictionary()
    all_classes = {}
    result = {}
    for i, line in enumerate(open( path + 'words.txt', 'r')):
        n_id, word = line.split('\t')[:2]
        all_classes[n_id] = word
    for key, value in id_dict.items():
        result[value] = (key, all_classes[key])
    return result

def get_data(id_dict):
    print('starting loading data')
    train_data, test_data = [], []
    train_labels, test_labels = [], []
    t = time.time()
    for key, value in id_dict.items():
        train_data += [nd.imread( path + 'train/{}/images/{}_{}.JPEG'.format(key, key, str(i)), mode='RGB') for i in range(500)]
        train_labels_ = np.array([[0]*200]*500)
        train_labels_[:, value] = 1
        train_labels += train_labels_.tolist()

    for line in open( path + 'val/val_annotations.txt'):
        img_name, class_id = line.split('\t')[:2]
        test_data.append(nd.imread( path + 'val/images/{}'.format(img_name) ,mode='RGB'))
        test_labels_ = np.array([[0]*200])
        test_labels_[0, id_dict[class_id]] = 1
        test_labels += test_labels_.tolist()

    print('finished loading data, in {} seconds'.format(time.time() - t))
    return np.array(train_data).astype('float32') / 255.0, np.array(train_labels), np.array(test_data).astype('float32') / 255.0, np.array(test_labels)

train_images, train_labels, test_images, test_labels = get_data(get_id_dictionary())

print( "train data shape: ",  train_images.shape )
print( "train label shape: ", train_labels.shape )
print( "test data shape: ",   test_images.shape )
print( "test_labels.shape: ", test_labels.shape )

train_images, test_images, train_labels, test_labels = train_test_split(train_images, train_labels, test_size=0.05, random_state=42)

fatal: destination path 'IMagenet' already exists and is not an empty directory.
test  train  val  wnids.txt  words.txt
starting loading data
finished loading data, in 44.53892779350281 seconds
train data shape:  (100000, 64, 64, 3)
train label shape:  (100000, 200)
test data shape:  (10000, 64, 64, 3)
test_labels.shape:  (10000, 200)


In [23]:
base_model = tf.keras.models.load_model('/content/model_configuration3.h5')

# Check its architecture
base_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 2, 2, 128)         0

In [19]:
base_model.pop()

In [20]:
base_model.trainable=False

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

model = tf.keras.models.Sequential()
model.add(MaxPooling2D(pool_size=(2, 2), input_shape=(64, 64, 3)))
model.add(base_model)
model.add(Flatten())
model.add(Dropout(.25))
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())

model.add(Dense(200, activation='softmax'))


model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])

history = model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_split=0.1)

test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test Accuracy (base model on TinyImagenet): {test_accuracy * 100:.2f}%')

Epoch 1/5
1336/1336 [==============================] - 10s 6ms/step - loss: 5.0911 - categorical_accuracy: 0.0223 - val_loss: 4.9447 - val_categorical_accuracy: 0.0339
Epoch 2/5
1336/1336 [==============================] - 7s 5ms/step - loss: 4.9680 - categorical_accuracy: 0.0312 - val_loss: 4.8988 - val_categorical_accuracy: 0.0387
Epoch 3/5
1336/1336 [==============================] - 7s 5ms/step - loss: 4.9335 - categorical_accuracy: 0.0348 - val_loss: 4.8835 - val_categorical_accuracy: 0.0396
Epoch 4/5
1336/1336 [==============================] - 7s 5ms/step - loss: 4.9128 - categorical_accuracy: 0.0358 - val_loss: 4.8793 - val_categorical_accuracy: 0.0420
Epoch 5/5
157/157 [==============================] - 1s 4ms/step - loss: 4.8866 - categorical_accuracy: 0.0376
Test Accuracy (base model on TinyImagenet): 3.76%


In [22]:
model.save('base_model_tiny_image.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
